In [ ]:
import os, math, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

from time import sleep

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
config.log_device_placement = True 
sess = tf.Session(config=config)
set_session(sess)


In [ ]:
def conv_bn_rl(x, f, k=1, s=1):
    x = layers.Conv2D(f, k, strides=s)(x)
    x = layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
    return x

def identity_block(tensor, f):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 1)
    x = layers.Conv2D(4*f, 1)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Add()([x, tensor])
    output =tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
    return output

def conv_block(tensor, f, s):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 1, s)
    x = layers.Conv2D(4*f, 1)(x)
    x = layers.BatchNormalization()(x)
    
    shortcut = layers.Conv2D(4*f, 1, strides=s)(tensor)
    shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Add()([x, shortcut])
    output =tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
    return output

def resnet_block(x, f, r, s=2):
    x = conv_block(x, f, s)
    for _ in range(r-1):
        x = identity_block(x, f)
    return x

def create_model():
    
    Input = layers.Input(shape=(224, 224, 3))
    x = conv_bn_rl(Input, 64, 7, 2)
    
    x = layers.MaxPool2D(3, strides=2)(x)

    x = resnet_block(x, 64, 3 , 1)
    x = resnet_block(x, 128, 4)
    x = resnet_block(x, 128, 6)
    x = resnet_block(x, 128, 3)
    
    x = layers.Conv2D(64, 3, strides=2)(x)
    x = layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
    
    x = layers.Conv2D(30, 3, strides=2)(x)
    
    x = tf.keras.layers.Reshape((2,15))(x)
    
    model = tf.keras.Model(inputs=Input, outputs=x)
    model.compile(optimizer=keras.optimizers.Adam(0.01), loss='mean_squared_error')
    return model

model=create_model()

model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
def read_tfrecord(example):
    features = {
        'image': tf.FixedLenFeature([], tf.string),
        'labels': tf.FixedLenFeature([], tf.string)
    }
    sample=tf.parse_single_example(example, features)
    image = tf.decode_raw(sample['image'], tf.uint8)
    image = tf.reshape(image, tf.stack([224, 224, 3]))
    image = augmentation(image)
    labels = tf.decode_raw(sample['labels'], tf.float64)
    labels = tf.reshape(labels, tf.stack([2,19]))
    labels = labels[:,:15] #remove head joints
    labels = tf.cast(labels, tf.float32)
    return image, labels/224.

def load_dataset(filenames):
    files = tf.data.Dataset.list_files(filenames)
    dataset = files.apply(tf.data.experimental.parallel_interleave(tf.data.TFRecordDataset, cycle_length=4))
    dataset = dataset.apply(tf.data.experimental.map_and_batch(map_func=read_tfrecord, batch_size=BATCH_SIZE, drop_remainder=True))
    dataset = dataset.apply(tf.data.experimental.shuffle_and_repeat(128, -1))
    dataset = dataset.prefetch(buffer_size=128)
    return dataset

def augmentation(img):
    image = tf.cast(img, tf.float32)
    image = tf.image.random_brightness(image, max_delta=25/255)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.per_image_standardization(image)
    return image

def get_batched_dataset(filenames):
    dataset = load_dataset(filenames)
    return dataset

def get_training_dataset():
    return get_batched_dataset(training_filenames)

def get_validation_dataset():
    return get_batched_dataset(validation_filenames)


In [ ]:
EPOCHS=10
name='model_grad_new'
BATCH_SIZE = 16
bucket_train = '/home/antreas/Desktop/tf_records_file/try/*'
bucket_valid = '/home/antreas/Desktop/tf_records_file/try/*'
train = tf.gfile.Glob(bucket_train)
valid = tf.gfile.Glob(bucket_valid)
training_filenames = train
validation_filenames = valid
steps_per_epoch = int(len(train)*1000/(BATCH_SIZE)) ###### to 1000 einai to plithos ton eikonon sto tfrecord #######
val_steps = int(len(valid)*1000/(BATCH_SIZE))


######## Tensorboard ############
logdir='/home/antreas/Desktop/model_feb/'+name
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True)
######## Model Save #############
filepath = '/home/antreas/Desktop/model_feb/'+name+'.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint((filepath),monitor='val_loss', save_best_only=True, verbose=1, save_weights_only=False)
#################################
clbk=[tensorboard_callback, checkpoint]

model=create_model()

model.fit(get_training_dataset(), validation_data=get_validation_dataset(), initial_epoch=0,
          steps_per_epoch=steps_per_epoch ,
          validation_steps=val_steps, epochs=EPOCHS, verbose=1, callbacks=clbk)


In [ ]:
# model = tf.keras.models.load_model('/home/antreas/Desktop/model_feb/model_grad_new.hdf5')

In [ ]:
train_images=np.load('/home/antreas/Desktop/test_images/RGB_IM_ARRAY.npy')##### Images

y=train_images[30]
im = np.expand_dims(y, axis=0)
x = model.predict(im)

x[0]

In [ ]:
i = x[0]*224
plt.imshow(y)
plt.plot(i[0][:], i[1][:] , '.' ,color='red')

In [ ]:
y=train_images[35]
plt.imshow[y]